# Data Preprocessing and Feature Engineering

## Overview

This notebook performs comprehensive data preprocessing and feature engineering following machine learning best practices. The key principle is to prevent data leakage by splitting the data early and using only training data to fit any transformers.

## Critical Workflow Order

1. **Data Cleaning** (Before Split) - Handle missing values, outliers, duplicates
2. **Train-Test Split** (Early) - Split data to prevent leakage
3. **Feature Engineering** (After Split) - Use only training data to fit transformers
4. **Apply Transformers** - Use fitted transformers on both training and test data
5. **Validation** - Ensure quality and no data leakage
6. **Save Results** - Store processed data for modeling

## Why This Order Matters

- **Data Leakage Prevention**: Test set remains completely unseen during feature engineering
- **Realistic Evaluation**: Model performance reflects real-world scenarios
- **Reproducibility**: All transformations are documented and saved
- **Best Practices**: Follows standard machine learning workflow

All preprocessing steps are configurable through `config.toml` for reproducibility and consistency.

In [ ]:
# Import required libraries for preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
from pathlib import Path
import warnings
from datetime import datetime

# Machine learning libraries
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from scipy import stats
from scipy.stats import boxcox

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration
import toml

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')

print("Libraries imported successfully!")
print("=" * 50)

# Load configuration
config = toml.load('../config.toml')

# Apply visualization settings from config
if 'visualization' in config:
    if 'dpi' in config['visualization']:
        plt.rcParams['figure.dpi'] = config['visualization']['dpi']
    if 'figure_size' in config['visualization']:
        plt.rcParams['figure.figsize'] = config['visualization']['figure_size']
    if 'color_palette' in config['visualization']:
        sns.set_palette(config['visualization']['color_palette'])

# Display preprocessing configuration
print("Preprocessing Configuration:")
print(f"Imputation strategy: {config['preprocessing']['imputation_strategy']}")
print(f"Outlier method: {config['preprocessing']['outlier_method']}")
print(f"Normalization method: {config['preprocessing']['normalization_method']}")
print(f"Test size: {config['split']['test_size']}")
print(f"Random state: {config['split']['random_state']}")
print(f"Stratify split: {config['split']['stratify']}")

## Loading Data and Previous Analysis Results

Before we begin preprocessing, we need to load:

1. **Original Data**: The raw dataset from the first notebook
2. **EDA Results**: Quality assessment and findings from the second notebook
3. **Metadata**: Information about columns and data structure

This ensures our preprocessing decisions are informed by the comprehensive analysis we performed earlier.

In [ ]:
import pickle
from pathlib import Path

print("Loading data from previous notebooks...")
print("=" * 50)

# Define data directory
data_dir = Path('../data/processed')

# Load data objects from data ingestion
with open(data_dir / '01_data_ingestion.pkl', 'rb') as f:
    ingestion_data = pickle.load(f)

# Load EDA results
with open(data_dir / '02_eda_results.pkl', 'rb') as f:
    eda_results = pickle.load(f)

# Extract data components
df = ingestion_data['data']['df'].copy()  # Make a copy to avoid modifying original
feature_cols = ingestion_data['data']['feature_cols']
sample_id_col = ingestion_data['metadata']['sample_id_col']
label_col = ingestion_data['metadata']['label_col']

# Extract metadata
n_features = ingestion_data['metadata']['n_features']
n_samples = ingestion_data['metadata']['n_samples']
label_distribution = ingestion_data['metadata']['label_distribution']

# Extract EDA results
quality_metrics = eda_results['quality_metrics']
recommendations = eda_results['recommendations']
quality_score = eda_results['quality_score']
correlation_matrix = eda_results['correlation_matrix']
high_corr_pairs = eda_results['high_corr_pairs']
outliers_by_feature = eda_results['outliers_by_feature']
class_distribution = eda_results['class_distribution']
feature_variances = eda_results['feature_variances']
analysis_timestamp = eda_results['analysis_timestamp']

# Summary printout
print("Data loaded successfully!")
print(f"Original dataset shape: {df.shape}")
print(f"Number of features: {n_features}")
print(f"Number of samples: {n_samples}")
print(f"Target variable: {label_col}")
print(f"Label distribution: {label_distribution}")

# EDA Summary
print(f"\nEDA Quality Score: {quality_score}/100")
print(f"Missing values: {quality_metrics['missing_values_percent']:.2f}%")
print(f"Outliers: {quality_metrics['outlier_percent']:.2f}%")
print(f"Class balance ratio: {quality_metrics['class_balance_ratio']:.3f}")

# Show EDA recommendations
print(f"\nEDA Recommendations ({len(recommendations)}):")
for i, rec in enumerate(recommendations, 1):
    print(f"  {i}. {rec}")

# Timestamp info
print(f"\nAnalysis performed at: {analysis_timestamp}")

## Preprocessing Plan

Based on our EDA findings and best practices, we'll implement the following preprocessing steps:

## Phase 1: Data Cleaning

### Step 1: Missing Value Treatment

**Why handle missing values before splitting?**

- Missing values can affect the split process
- Basic cleaning should be done on the entire dataset
- No risk of data leakage for simple imputation

**Imputation Strategies:**

- **Mean**: Replace with feature mean (good for normally distributed data)
- **Median**: Replace with feature median (robust to outliers)
- **Most Frequent**: Replace with most common value (good for categorical data)
- **KNN**: Use k-nearest neighbors to estimate missing values
- **None**: Remove rows with missing values (if missing data is minimal)

**Our Approach:**

1. Analyze missing value patterns
2. Choose appropriate imputation strategy from config
3. Apply imputation to features and handle target separately
4. Validate imputation results

In [ ]:
# Phase 1: Data Cleaning
print("Phase 1: Data Cleaning")
print("=" * 50)

# Step 1: Missing Value Treatment
print("Step 1: Missing Value Treatment")
print("-" * 30)

# Analyze missing values
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

print("Missing Value Analysis:")
print(f"Total missing values: {missing_data.sum()}")
print(f"Overall missing percentage: {missing_data.sum() / (len(df) * len(df.columns)) * 100:.2f}%")

# Show features with missing values
features_with_missing = missing_data[missing_data > 0]
if len(features_with_missing) > 0:
    print(f"\nFeatures with missing values ({len(features_with_missing)}):")
    for feature, count in features_with_missing.items():
        print(f"  {feature}: {count} ({missing_percent[feature]:.2f}%)")
else:
    print("\n✓ No missing values found in features.")

# Check target variable for missing values
if df[label_col].isnull().sum() > 0:
    print(f"\n⚠️  Missing values in target variable: {df[label_col].isnull().sum()}")
    # Remove rows with missing target (we can't impute the target)
    df = df.dropna(subset=[label_col])
    print(f"Removed {len(df)} rows with missing target values")
else:
    print(f"\n✓ No missing values in target variable")

# Apply imputation strategy
imputation_strategy = config['preprocessing']['imputation_strategy']
print(f"\nApplying imputation strategy: {imputation_strategy}")

if imputation_strategy != "none" and len(features_with_missing) > 0:
    # Prepare feature data for imputation
    X_features = df[feature_cols].copy()
    
    if imputation_strategy == "mean":
        imputer = SimpleImputer(strategy='mean')
        print("Using mean imputation (average value for each feature)")
        
    elif imputation_strategy == "median":
        imputer = SimpleImputer(strategy='median')
        print("Using median imputation (middle value for each feature)")
        
    elif imputation_strategy == "most_frequent":
        imputer = SimpleImputer(strategy='most_frequent')
        print("Using most frequent imputation (most common value for each feature)")
        
    elif imputation_strategy == "knn":
        imputer = KNNImputer(n_neighbors=5)
        print("Using KNN imputation (k=5 nearest neighbors)")
    
    # Fit and transform
    X_imputed = imputer.fit_transform(X_features)
    df[feature_cols] = X_imputed
    
    print("✓ Imputation completed successfully")
    
    # Save imputer for later use
    imputer_info = {
        'imputer': imputer,
        'strategy': imputation_strategy,
        'features_imputed': list(features_with_missing.index)
    }
    
else:
    print("✓ No imputation needed (no missing values or strategy is 'none')")
    imputer_info = None

# Validate imputation
missing_after = df.isnull().sum().sum()
print(f"\nValidation:")
print(f"Missing values after imputation: {missing_after}")
if missing_after == 0:
    print("✓ All missing values have been successfully handled")
else:
    print(f"⚠️  {missing_after} missing values remain")

print(f"Dataset shape after missing value treatment: {df.shape}")

### Step 2: Outlier Detection and Treatment

**Why handle outliers before splitting?**

- Outliers can affect the split process and class distribution
- Basic outlier treatment doesn't involve complex statistics
- No risk of data leakage for simple outlier methods

**Outlier Detection Methods:**

- **IQR Method**: Uses interquartile range (Q3 - Q1) to identify outliers
- **Z-Score Method**: Uses standard deviations from the mean
- **Isolation Forest**: Machine learning approach to detect outliers
- **None**: Skip outlier detection

**Treatment Strategies:**

- **Remove**: Delete outlier samples (if percentage is low)
- **Cap**: Limit outliers to reasonable bounds
- **Investigate**: Flag for manual review (if percentage is high)

In [ ]:
# Step 2: Outlier Detection and Treatment
print("\nStep 2: Outlier Detection and Treatment")
print("-" * 40)

outlier_method = config['preprocessing']['outlier_method']
print(f"Using outlier detection method: {outlier_method}")

# Prepare data for outlier detection
X_features = df[feature_cols].copy()

outlier_info = {
    'method': outlier_method,
    'outliers_by_feature': {},
    'total_outliers': 0,
    'outlier_indices': set()
}

if outlier_method == "iqr":
    print("Using IQR method for outlier detection")
    print("Outliers: values < Q1 - 1.5*IQR or > Q3 + 1.5*IQR")
    
    for col in feature_cols:
        Q1 = X_features[col].quantile(0.25)
        Q3 = X_features[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = ((X_features[col] < lower_bound) | (X_features[col] > upper_bound))
        outlier_count = outliers.sum()
        outlier_info['outliers_by_feature'][col] = outlier_count
        outlier_info['total_outliers'] += outlier_count
        
        if outlier_count > 0:
            outlier_indices = outliers[outliers].index
            outlier_info['outlier_indices'].update(outlier_indices)
            
            print(f"  {col}: {outlier_count} outliers ({outlier_count/len(df)*100:.1f}%)")

elif outlier_method == "zscore":
    print("Using Z-score method for outlier detection")
    print("Outliers: values with |z-score| > 3")
    
    z_scores = np.abs(stats.zscore(X_features))
    threshold = 3.0
    
    for i, col in enumerate(feature_cols):
        outliers = z_scores[:, i] > threshold
        outlier_count = outliers.sum()
        outlier_info['outliers_by_feature'][col] = outlier_count
        outlier_info['total_outliers'] += outlier_count
        
        if outlier_count > 0:
            outlier_indices = np.where(outliers)[0]
            outlier_info['outlier_indices'].update(outlier_indices)
            
            print(f"  {col}: {outlier_count} outliers ({outlier_count/len(df)*100:.1f}%)")

elif outlier_method == "isolation_forest":
    print("Using Isolation Forest for outlier detection")
    
    iso_forest = IsolationForest(contamination=0.1, random_state=42)
    outlier_labels = iso_forest.fit_predict(X_features)
    outliers = outlier_labels == -1
    outlier_count = outliers.sum()
    
    outlier_info['total_outliers'] = outlier_count
    outlier_info['outlier_indices'] = set(np.where(outliers)[0])
    
    print(f"  Total outliers detected: {outlier_count} ({outlier_count/len(df)*100:.1f}%)")

else:
    print("Skipping outlier detection (method: none)")

# Analyze outlier impact and decide treatment
if outlier_info['total_outliers'] > 0:
    outlier_percentage = len(outlier_info['outlier_indices']) / len(df) * 100
    print(f"\nOutlier Analysis:")
    print(f"Total unique samples with outliers: {len(outlier_info['outlier_indices'])}")
    print(f"Percentage of samples with outliers: {outlier_percentage:.1f}%")
    
    # Decide on treatment strategy
    if outlier_percentage > 20:
        print("⚠️  High outlier percentage - will cap outliers")
        treatment_strategy = "cap"
    elif outlier_percentage > 5:
        print("Moderate outlier percentage - will cap outliers")
        treatment_strategy = "cap"
    else:
        print("Low outlier percentage - will remove outliers")
        treatment_strategy = "remove"
    
    # Apply treatment
    if treatment_strategy == "remove":
        print("\nRemoving outlier samples...")
        df_clean = df.drop(index=list(outlier_info['outlier_indices']))
        removed_count = len(df) - len(df_clean)
        df = df_clean
        print(f"Removed {removed_count} samples with outliers")
        
    elif treatment_strategy == "cap":
        print("\nCapping outliers to 95th and 5th percentiles...")
        for col in feature_cols:
            if outlier_info['outliers_by_feature'].get(col, 0) > 0:
                lower_bound = X_features[col].quantile(0.05)
                upper_bound = X_features[col].quantile(0.95)
                df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
        print("✓ Outliers capped successfully")
    
    outlier_info['treatment_strategy'] = treatment_strategy
    outlier_info['outlier_percentage'] = outlier_percentage
    
else:
    print("✓ No outliers detected")
    outlier_info['treatment_strategy'] = "none"
    outlier_info['outlier_percentage'] = 0

print(f"Dataset shape after outlier treatment: {df.shape}")

In [ ]:
# Step 3: Duplicate Removal
print("\nStep 3: Duplicate Removal")
print("-" * 25)

# Check for duplicate rows
duplicate_rows = df.duplicated().sum()
duplicate_percent = (duplicate_rows / len(df)) * 100

print(f"Duplicate rows: {duplicate_rows} ({duplicate_percent:.2f}%)")

if duplicate_rows > 0:
    print("Removing duplicate rows...")
    df = df.drop_duplicates()
    print(f"Removed {duplicate_rows} duplicate rows")
else:
    print("✓ No duplicate rows found")

# Check for duplicate sample IDs (if present)
if sample_id_col and sample_id_col in df.columns:
    duplicate_ids = df[sample_id_col].duplicated().sum()
    if duplicate_ids > 0:
        print(f"Duplicate sample IDs: {duplicate_ids}")
        print("Removing rows with duplicate sample IDs...")
        df = df.drop_duplicates(subset=[sample_id_col])
        print(f"Removed {duplicate_ids} rows with duplicate sample IDs")
    else:
        print("✓ No duplicate sample IDs found")

print(f"Dataset shape after duplicate removal: {df.shape}")

# Summary of cleaning phase
print(f"\nPhase 1 Summary:")
print(f"Original shape: {ingestion_data['data']['df'].shape}")
print(f"After cleaning: {df.shape}")
print(f"Rows removed: {ingestion_data['data']['df'].shape[0] - df.shape[0]}")

## Phase 2: Train-Test Split

### Why Split Early?

**Critical Principle**: We split the data BEFORE any feature engineering to prevent data leakage.

**What happens if we don't split early?**

- Feature selection would use information from test set
- Normalization would be influenced by test set statistics
- Model evaluation would be overly optimistic
- Results wouldn't reflect real-world performance

**Our Approach:**

1. Split data after basic cleaning
2. Use only training data for all feature engineering
3. Apply fitted transformers to test data
4. Maintain proper separation throughout

### Split Strategy

- **Test Size**: Percentage from config (typically 20%)
- **Random State**: Fixed seed for reproducibility
- **Stratification**: Maintain class distribution in both sets

In [ ]:
# Phase 2: Train-Test Split
print("Phase 2: Train-Test Split")
print("=" * 50)

# Get split parameters from config
test_size = config['split']['test_size']
random_state = config['split']['random_state']
stratify = config['split']['stratify']

print(f"Split parameters:")
print(f"  Test size: {test_size}")
print(f"  Random state: {random_state}")
print(f"  Stratify: {stratify}")

# Prepare data for splitting
X = df[feature_cols]
y = df[label_col]

print(f"\nData prepared for splitting:")
print(f"  Features shape: {X.shape}")
print(f"  Target shape: {y.shape}")

# Perform train-test split
if stratify:
    print("\nPerforming stratified split...")
    X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
        X, y, range(len(df)), 
        test_size=test_size, 
        random_state=random_state, 
        stratify=y
    )
else:
    print("\nPerforming random split...")
    X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
        X, y, range(len(df)), 
        test_size=test_size, 
        random_state=random_state
    )

# Create train and test dataframes
train_df = df.iloc[train_indices].copy()
test_df = df.iloc[test_indices].copy()

print(f"\nSplit results:")
print(f"  Training set: {len(train_df)} samples ({len(train_df)/len(df)*100:.1f}%)")
print(f"  Test set: {len(test_df)} samples ({len(test_df)/len(df)*100:.1f}%)")

# Verify class distribution in splits
print(f"\nClass distribution in training set:")
train_class_dist = train_df[label_col].value_counts()
for class_name, count in train_class_dist.items():
    print(f"  {class_name}: {count} ({count/len(train_df)*100:.1f}%)")

print(f"\nClass distribution in test set:")
test_class_dist = test_df[label_col].value_counts()
for class_name, count in test_class_dist.items():
    print(f"  {class_name}: {count} ({count/len(test_df)*100:.1f}%)")

# Check if stratification worked
if stratify:
    original_dist = df[label_col].value_counts(normalize=True)
    train_dist = train_df[label_col].value_counts(normalize=True)
    test_dist = test_df[label_col].value_counts(normalize=True)
    
    print(f"\nStratification check:")
    for class_name in original_dist.index:
        orig_pct = original_dist[class_name] * 100
        train_pct = train_dist[class_name] * 100
        test_pct = test_dist[class_name] * 100
        print(f"  {class_name}: Original={orig_pct:.1f}%, Train={train_pct:.1f}%, Test={test_pct:.1f}%")

# Save split information
split_info = {
    'test_size': test_size,
    'random_state': random_state,
    'stratify': stratify,
    'train_indices': train_indices,
    'test_indices': test_indices,
    'train_shape': train_df.shape,
    'test_shape': test_df.shape,
    'train_class_distribution': train_class_dist.to_dict(),
    'test_class_distribution': test_class_dist.to_dict()
}

print(f"\n✓ Train-test split completed successfully")
print("⚠️  IMPORTANT: From now on, we will only use training data to fit any transformers!")

## Phase 3: Feature Engineering

### Critical Principle: Training Data Only

**Why use only training data for feature engineering?**

- **No Data Leakage**: Test set remains completely unseen
- **Realistic Evaluation**: Model performance reflects real-world scenarios
- **Proper ML Workflow**: Follows standard machine learning best practices

**Our Approach:**

1. **Fit transformers on training data only**
2. **Transform training data** using fitted transformers
3. **Transform test data** using the SAME fitted transformers
4. **Never refit transformers** on test data

### Feature Engineering Steps

1. **Feature Selection**: Remove highly correlated and low-variance features
2. **Feature Transformation**: Apply normalization and handle skewed distributions
3. **Categorical Encoding**: Handle categorical variables if present

### Example of Proper Workflow
```python
# CORRECT approach:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit on training
X_test_scaled = scaler.transform(X_test)        # Apply to test

# WRONG approach (data leakage):
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all)      # Uses test data info
```

Let's implement this step by step.

In [ ]:
# Phase 3: Feature Engineering
print("Phase 3: Feature Engineering")
print("=" * 50)

print("⚠️  REMINDER: All feature engineering will use training data only!")
print("-" * 60)

# Step 1: Feature Selection - Correlation Removal
print("Step 1: Feature Selection - Correlation Removal")
print("-" * 50)

# Calculate correlation matrix using TRAINING DATA ONLY
print("Calculating correlation matrix using training data only...")
train_correlation_matrix = train_df[feature_cols].corr()

# Find highly correlated features using training data
correlation_threshold = 0.8  # Can be made configurable
high_corr_pairs = []

for i in range(len(train_correlation_matrix.columns)):
    for j in range(i+1, len(train_correlation_matrix.columns)):
        corr_val = train_correlation_matrix.iloc[i, j]
        if abs(corr_val) > correlation_threshold:
            high_corr_pairs.append((
                train_correlation_matrix.columns[i],
                train_correlation_matrix.columns[j],
                corr_val
            ))

print(f"Features with correlation > {correlation_threshold}: {len(high_corr_pairs)}")

if high_corr_pairs:
    print("\nHighly correlated feature pairs (first 10):")
    for feat1, feat2, corr in sorted(high_corr_pairs, key=lambda x: abs(x[2]), reverse=True)[:10]:
        print(f"  {feat1} - {feat2}: {corr:.3f}")

# Find features to remove based on high correlation
high_corr_features = set()

for feat1, feat2, corr in high_corr_pairs:
    # Keep the feature with higher variance (more information)
    var1 = train_df[feat1].var()
    var2 = train_df[feat2].var()
    
    if var1 >= var2:
        high_corr_features.add(feat2)
    else:
        high_corr_features.add(feat1)

print(f"\nFeatures to remove due to high correlation: {len(high_corr_features)}")
if high_corr_features:
    print("Removing:", list(high_corr_features)[:10])  # Show first 10

# Remove highly correlated features
feature_cols_clean = [col for col in feature_cols if col not in high_corr_features]
print(f"Features after correlation removal: {len(feature_cols_clean)}")

# Update feature columns for both training and test sets
X_train_clean = train_df[feature_cols_clean]
X_test_clean = test_df[feature_cols_clean]

print(f"Training features shape: {X_train_clean.shape}")
print(f"Test features shape: {X_test_clean.shape}")

In [ ]:
# Step 2: Feature Selection - Variance Selection
print("\nStep 2: Feature Selection - Variance Selection")
print("-" * 45)

# Remove low-variance features using TRAINING DATA ONLY
variance_threshold = 0.01  # Can be made configurable
variance_selector = VarianceThreshold(threshold=variance_threshold)

print(f"Using variance threshold: {variance_threshold}")
print("Fitting variance selector on training data only...")

# Fit variance selector on training data only
variance_selector.fit(X_train_clean)

# Get selected features
selected_features_mask = variance_selector.get_support()
low_variance_features = [col for col, selected in zip(feature_cols_clean, selected_features_mask) if not selected]

print(f"Low-variance features removed: {len(low_variance_features)}")
if low_variance_features:
    print("Removed:", low_variance_features[:10])  # Show first 10

# Get final feature list
final_features = [col for col, selected in zip(feature_cols_clean, selected_features_mask) if selected]
print(f"Final features after selection: {len(final_features)}")

# Apply feature selection to both datasets
X_train_selected = X_train_clean[final_features]
X_test_selected = test_df[final_features]  # Use original test_df to get final features

print(f"Training features after selection: {X_train_selected.shape}")
print(f"Test features after selection: {X_test_selected.shape}")

# Save feature selection info
feature_selection_info = {
    'original_features': len(feature_cols),
    'features_after_correlation_removal': len(feature_cols_clean),
    'features_after_variance_selection': len(final_features),
    'removed_correlated_features': list(high_corr_features),
    'removed_low_variance_features': low_variance_features,
    'final_features': final_features,
    'correlation_threshold': correlation_threshold,
    'variance_threshold': variance_threshold
}

print(f"\nFeature Selection Summary:")
print(f"Original features: {feature_selection_info['original_features']}")
print(f"After correlation removal: {feature_selection_info['features_after_correlation_removal']}")
print(f"After variance selection: {feature_selection_info['features_after_variance_selection']}")
print(f"Total features removed: {len(high_corr_features) + len(low_variance_features)}")

In [ ]:
# Step 3: Feature Transformation - Normalization
print("\nStep 3: Feature Transformation - Normalization")
print("-" * 50)

normalization_method = config['preprocessing']['normalization_method']
print(f"Using normalization method: {normalization_method}")

# Initialize scaler based on config
if normalization_method == "zscore":
    scaler = StandardScaler()
    print("Using Z-score normalization (mean=0, std=1)")
    print("Formula: (x - mean) / std")
    
elif normalization_method == "minmax":
    scaler = MinMaxScaler()
    print("Using Min-Max normalization (range [0,1])")
    print("Formula: (x - min) / (max - min)")
    
elif normalization_method == "robust":
    scaler = RobustScaler()
    print("Using Robust normalization (median=0, IQR=1)")
    print("Formula: (x - median) / IQR")
    
else:
    print("Skipping normalization")
    scaler = None

# Apply normalization if specified
if scaler is not None:
    print("\nFitting scaler on training data only...")
    print("This is the critical step that prevents data leakage!")
    
    # Fit scaler on training data only
    X_train_scaled = scaler.fit_transform(X_train_selected)
    
    # Apply the SAME fitted scaler to test data
    X_test_scaled = scaler.transform(X_test_selected)
    
    print("✓ Normalization completed successfully")
    print("✓ Scaler fitted on training data only")
    print("✓ Same scaler applied to test data")
    
    # Show sample of normalized data
    print("\nSample of normalized training data:")
    print(pd.DataFrame(X_train_scaled, columns=final_features).head())
    
    # Show scaler parameters (from training data)
    if hasattr(scaler, 'mean_'):
        print(f"\nScaler parameters (from training data):")
        print(f"Mean: {scaler.mean_[:5]}...")  # Show first 5
        print(f"Scale: {scaler.scale_[:5]}...")  # Show first 5
    
else:
    print("✓ No normalization applied")
    X_train_scaled = X_train_selected.values
    X_test_scaled = X_test_selected.values

# Save normalization info
normalization_info = {
    'method': normalization_method,
    'scaler': scaler,
    'features_normalized': final_features
}

print(f"\nNormalization Summary:")
print(f"Method: {normalization_method}")
print(f"Features normalized: {len(final_features)}")
print(f"Training data shape: {X_train_scaled.shape}")
print(f"Test data shape: {X_test_scaled.shape}")

## Phase 4: Data Validation

### What We're Validating

1. **Data Completeness**: No missing values
2. **Data Types**: Correct data types for each column
3. **Value Ranges**: Features are within expected ranges
4. **Distribution Changes**: How preprocessing affected data distributions
5. **Data Leakage Check**: Ensure test set wasn't used for fitting
6. **Statistical Properties**: Key statistics before and after preprocessing

### Validation Checks

- **Missing Values**: Ensure no missing values remain
- **Data Types**: Verify correct data types
- **Value Ranges**: Check for reasonable value ranges
- **Distribution Analysis**: Compare before/after distributions
- **Statistical Summary**: Key statistics and changes
- **Data Leakage Verification**: Confirm proper workflow

In [ ]:
# Phase 4: Data Validation
print("Phase 4: Data Validation")
print("=" * 50)

# 4.1 Check for missing values
print("4.1 Checking for missing values...")
train_missing = np.isnan(X_train_scaled).sum()
test_missing = np.isnan(X_test_scaled).sum()

if train_missing == 0 and test_missing == 0:
    print("✓ No missing values found in processed data")
else:
    print(f"⚠️  Missing values found: Train={train_missing}, Test={test_missing}")

# 4.2 Check data types and value ranges
print("\n4.2 Checking data types and value ranges...")

# Check for infinite values
train_infinite = np.isinf(X_train_scaled).sum()
test_infinite = np.isinf(X_test_scaled).sum()

if train_infinite == 0 and test_infinite == 0:
    print("✓ No infinite values found")
else:
    print(f"⚠️  Infinite values found: Train={train_infinite}, Test={test_infinite}")

# Check value ranges
print("\nValue ranges (first 5 features):")
for i, feature in enumerate(final_features[:5]):
    train_min, train_max = X_train_scaled[:, i].min(), X_train_scaled[:, i].max()
    test_min, test_max = X_test_scaled[:, i].min(), X_test_scaled[:, i].max()
    print(f"  {feature}: Train[{train_min:.3f}, {train_max:.3f}], Test[{test_min:.3f}, {test_max:.3f}]")

# 4.3 Distribution comparison (before vs after)
print("\n4.3 Distribution analysis...")

# Create comparison plots for first 3 features
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
original_train_data = train_df[final_features[:3]]

for i, feature in enumerate(final_features[:3]):
    # Before preprocessing
    axes[0, i].hist(original_train_data[feature], bins=30, alpha=0.7, color='blue')
    axes[0, i].set_title(f'{feature} (Before)')
    axes[0, i].set_xlabel('Value')
    
    # After preprocessing
    axes[1, i].hist(X_train_scaled[:, i], bins=30, alpha=0.7, color='green')
    axes[1, i].set_title(f'{feature} (After)')
    axes[1, i].set_xlabel('Value')

plt.tight_layout()
plt.show()

# 4.4 Statistical summary comparison
print("\n4.4 Statistical summary comparison:")
print("Before preprocessing (first 3 features):")
print(original_train_data.describe())

print("\nAfter preprocessing (first 3 features):")
processed_df = pd.DataFrame(X_train_scaled[:, :3], columns=final_features[:3])
print(processed_df.describe())

# 4.5 Data leakage verification
print("\n4.5 Data leakage verification...")
print("✓ Train-test split performed before feature engineering")
print("✓ Feature selection used only training data")
print("✓ Normalization fitted on training data only")
print("✓ Test data transformed using fitted transformers")
print("✓ No information from test set influenced preprocessing")

# 4.6 Quality metrics
print("\n4.6 Quality metrics:")
validation_metrics = {
    'train_shape': X_train_scaled.shape,
    'test_shape': X_test_scaled.shape,
    'feature_count': len(final_features),
    'train_missing': train_missing,
    'test_missing': test_missing,
    'train_infinite': train_infinite,
    'test_infinite': test_infinite
}

for metric, value in validation_metrics.items():
    print(f"  {metric}: {value}")

# Overall validation score
validation_score = 100
if train_missing > 0 or test_missing > 0:
    validation_score -= 20
if train_infinite > 0 or test_infinite > 0:
    validation_score -= 20

print(f"\nValidation Score: {validation_score}/100")
if validation_score >= 80:
    print("✓ Data validation passed")
else:
    print("⚠️  Data validation issues detected")

## Phase 5: Save Results

### What We're Saving

We'll save all the processed data and preprocessing information for use in subsequent notebooks:

1. **Processed Datasets**: Clean training and test sets
2. **Preprocessing Pipeline**: All transformers and parameters
3. **Feature Information**: Selected features and metadata
4. **Split Information**: Train-test split details
5. **Preprocessing Report**: Summary of all steps applied

### Why Save Everything?

1. **Reproducibility**: Ensure consistent preprocessing across runs
2. **Pipeline Continuity**: Next notebooks can use the same preprocessing
3. **Documentation**: Record all decisions and transformations
4. **Debugging**: Easy to trace issues back to preprocessing steps

### File Structure

- `processed_data.pkl`: Training and test datasets
- `preprocessing_pipeline.pkl`: All transformers and parameters
- `feature_info.pkl`: Feature selection and metadata
- `preprocessing_summary.json`: Human-readable summary

In [ ]:
# Phase 5: Save Results
print("Phase 5: Save Results")
print("=" * 50)

# Create output directory
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# 1. Save processed datasets
processed_data = {
    'X_train': X_train_scaled,
    'X_test': X_test_scaled,
    'y_train': y_train,
    'y_test': y_test,
    'feature_cols': final_features,
    'label_col': label_col,
    'sample_id_col': sample_id_col,
    'train_indices': train_indices,
    'test_indices': test_indices
}

with open(output_dir / '03_processed_data.pkl', 'wb') as f:
    pickle.dump(processed_data, f)
print("✓ Processed datasets saved")

# 2. Save preprocessing pipeline
preprocessing_pipeline = {
    'imputer_info': imputer_info,
    'outlier_info': outlier_info,
    'feature_selection_info': feature_selection_info,
    'normalization_info': normalization_info,
    'split_info': split_info,
    'validation_metrics': validation_metrics,
    'validation_score': validation_score
}

with open(output_dir / '03_preprocessing_pipeline.pkl', 'wb') as f:
    pickle.dump(preprocessing_pipeline, f)
print("✓ Preprocessing pipeline saved")

# 3. Save feature information
feature_info = {
    'original_features': feature_cols,
    'final_features': final_features,
    'removed_correlated': feature_selection_info['removed_correlated_features'],
    'removed_low_variance': feature_selection_info['removed_low_variance_features']
}

with open(output_dir / '03_feature_info.pkl', 'wb') as f:
    pickle.dump(feature_info, f)
print("✓ Feature information saved")

# 4. Create preprocessing summary
preprocessing_summary = {
    'timestamp': datetime.now().isoformat(),
    'dataset_info': {
        'original_shape': ingestion_data['data']['df'].shape,
        'cleaned_shape': df.shape,
        'train_shape': X_train_scaled.shape,
        'test_shape': X_test_scaled.shape
    },
    'preprocessing_steps': {
        'imputation': {
            'strategy': imputation_strategy,
            'features_imputed': len(features_with_missing) if features_with_missing is not None else 0
        },
        'outlier_treatment': {
            'method': outlier_method,
            'outliers_found': outlier_info['total_outliers'],
            'treatment_strategy': outlier_info.get('treatment_strategy', 'none')
        },
        'feature_selection': {
            'original_features': feature_selection_info['original_features'],
            'final_features': len(final_features),
            'correlation_removal': len(feature_selection_info['removed_correlated_features']),
            'variance_selection': len(feature_selection_info['removed_low_variance_features'])
        },
        'normalization': {
            'method': normalization_method,
            'features_normalized': len(final_features)
        }
    },
    'quality_metrics': {
        'validation_score': validation_score,
        'train_missing': train_missing,
        'test_missing': test_missing,
        'train_infinite': train_infinite,
        'test_infinite': test_infinite
    },
    'split_info': {
        'test_size': test_size,
        'stratify': stratify,
        'train_samples': X_train_scaled.shape[0],
        'test_samples': X_test_scaled.shape[0]
    },
    'data_leakage_prevention': {
        'split_before_feature_engineering': True,
        'feature_selection_on_training_only': True,
        'normalization_fitted_on_training_only': True,
        'test_data_unseen_during_preprocessing': True
    }
}

with open(output_dir / '03_preprocessing_summary.pkl', 'wb') as f:
    pickle.dump(preprocessing_summary, f)
print("✓ Preprocessing summary saved")